In [ ]:
using Polynomials,SpecialFunctions
using LinearAlgebra
using NLsolve
using Plots,LaTeXStrings
default(markersize=3,linewidth=1.5)
#include("FNC.jl")
include("functions/chapter04.jl");

# Example 4.1.1

In the theory of vibrations of a circular drum, the displacement of the drumhead can be expressed in terms of pure harmonic modes, 

$$J_m(\omega_{k,m} r) \cos(m\theta) \cos(c \omega_{k,m} t),$$

where $(r,\theta)$ are polar coordinates, $0\le r\le 1$, $t$ is time, $m$ is a positive integer, $c$ is a material parameter, and $J_m$ is a _Bessel function of the first kind_. The quantity $\omega_{k,m}$ is a resonant frequency and is a positive root of the equation  

$$J_m(\omega_{k,m}) = 0,$$ 

which states that the drumhead is clamped around the rim. Tabulating approximations to the zeros of Bessel functions has occupied countless mathematician-hours throughout the centuries. 

In [ ]:
J3(x) = besselj(3,x)
plot(J3,0,20,
    grid=:xy,legend=:none,
    xaxis=(L"x"),yaxis=(L"J_3(x)"),title="Bessel function")

From the graph we see roots near 6, 10, 13, 16, and 19. We use `nlsolve` from the `NLsolve` package to find these roots accurately. (It uses vector variables, so we have to adapt it for use with scalars.)

In [ ]:
methods(nlsolve)

In [ ]:
omega = Float64[]
for guess = [6.,10.,13.,16.,19.]
    s = nlsolve(x->besselj(3,x[1]),[guess])
    omega = push!(omega,s.zero[1])
end
omega

In [ ]:
scatter!(omega,J3.(omega),title="Bessel function with roots")

# Example 4.3.1

Suppose we want to find a root of the function 

In [ ]:
f = x -> x*exp(x) - 2

plot(f,0,1.5,label="function",grid=:y,
    xlabel="x",ylabel="y",legend=:topleft)

From the graph, it is clear that there is a root near $x=1$. So we call that our initial guess, $x_1$.

In [ ]:
x1 = 1
f1 = f(x1)
scatter!([x1],[f1],label="initial point")

Next, we can compute the tangent line at the point $\bigl(x_1,f(x_1)\bigr)$, using the derivative.

In [ ]:
dfdx = x -> exp(x)*(x+1)
slope1 = dfdx(x1)
tangent1 = x -> f1 + slope1*(x-x1)

plot!(tangent1,0,1.5,l=:dash,label="tangent line",ylim=[-2,4])

In lieu of finding the root of $f$ itself, we settle for finding the root of the tangent line approximation, which is trivial. Call this $x_2$, our next approximation to the root.

In [ ]:
x2 = x1 - f1/slope1
scatter!([x2],[0],label="tangent root")

In [ ]:
f2 = f(x2)

The residual (value of $f$) is smaller than before, but not zero. So we repeat the process with a new tangent line based on the latest point on the curve.

In [ ]:
plot(f,0.83,0.88)
scatter!([x2],[f2])
slope2 = dfdx(x2)
tangent2 = x -> f2 + slope2*(x-x2)
plot!(tangent2,0.83,0.88,l=:dash)
x3 = x2 - f2/slope2
scatter!([x3],[0],
    legend=:none,xlabel="x",ylabel="y",title="Second iteration")

In [ ]:
f1

In [ ]:
f2

In [ ]:
f3 = f(x3)

We appear to be getting closer to the true root each time. 

# Example 4.3.2

We again look at finding a solution of $xe^x=2$ near $x=1$. To apply Newton's method, we need to calculate values of both the residual function $f$ and its derivative.  

In [ ]:
f = x -> x*exp(x) - 2;
dfdx = x -> exp(x)*(x+1);

We don't know the exact root, so we use `nlsolve` (from `NLsolve`) to determine the "true" value.

In [ ]:
r = nlsolve(x -> f(x[1]),[1.]).zero

We use $x_1=1$ as a starting guess and apply the iteration in a loop, storing the sequence of iterates in a vector.

In [ ]:
x = [1;zeros(6)]
for k = 1:6
    x[k+1] = x[k] - f(x[k]) / dfdx(x[k])
end
x

Here is the sequence of errors. 

In [ ]:
err = @. x - r

Glancing at the exponents of the errors, they roughly form a neat doubling sequence until the error is comparable to machine precision. We can see this more precisely by taking logs.

In [ ]:
logerr = @. log(abs(err))

Quadratic convergence isn't as graphically distinctive as linear convergence.

In [ ]:
plot(0:6,abs.(err),m=:o,label="",
    xlabel="\$k\$",yaxis=(:log10,"\$|x_k-r|\$"),title="Quadratic convergence")

This looks faster than linear convergence, but it's not easy to say more. If we could use infinite precision, the curve would continue to steepen forever.

# Example 4.3.3

Suppose we want to solve $e^x=x+c$ for multiple values of $c$. We can create functions for $f$ and $f'$ in each case.

In [ ]:
for c = [2,4,7.5,11]
    f = x -> exp(x) - x - c;
    dfdx = x -> exp(x) - 1;
    x = newton(f,dfdx,1.0);  r = x[end];
    println("root with c = $c is $r")
end

There's a subtlety about the definition of `f`. It uses whatever value is assigned to `c` at the moment `f` is called. (This is unlike MATLAB, which locks in the value defined for `c` at the moment of definition.) If we later change the value assigned to `c`, the function is changed also.

In [ ]:
c = 11;  f = x -> exp(x) - x - c;
@show f(0);

In [ ]:
c = 100; 
@show f(0);

# Example 4.4.1

We return to finding a root of the equation $xe^x=2$.

In [ ]:
f = x -> x*exp(x) - 2;
plot(f,0.25,1.25,label="function",leg=:topleft)

From the graph, it's clear that there is a root near $x=1$. To be more precise, there is a root in the interval $[0.5,1]$. So let us take the endpoints of that interval as _two_ initial approximations. 

In [ ]:
x1 = 1;    f1 = f(x1);
x2 = 0.5;  f2 = f(x2);
scatter!([x1,x2],[f1,f2],label="initial points")

Instead of constructing the tangent line by evaluating the derivative, we can construct a linear model function by drawing the line between the two points $\bigl(x_1,f(x_1)\bigr)$ and $\bigl(x_2,f(x_2)\bigr)$. This is called a _secant line_.

In [ ]:
slope2 = (f2-f1) / (x2-x1);
secant2 = x -> f2 + slope2*(x-x2);
plot!(secant2,0.25,1.25,label="secant line",l=:dash,color=:black)

As before, the next value in the iteration is the root of this linear model. 

In [ ]:
x3 = x2 - f2/slope2;
@show f3 = f(x3)
scatter!([x3],[0],label="root of secant")

For the next linear model, we use the line through the two most recent points. The next iterate is the root of that secant line, and so on.

In [ ]:
slope3 = (f3-f2) / (x3-x2);
x4 = x3 - f3/slope3;
f4 = f(x4)

# Example 4.4.2

We check the convergence of the secant method from the previous example.

In [ ]:
f = x -> x*exp(x) - 2;
x = secant(f,1,0.5)

We don't know the exact root, so we use `nlsolve` to get a substitute.

In [ ]:
r = nlsolve(x->f(x[1]),[1.]).zero

Here is the sequence of errors. 

In [ ]:
err = @. r - x

It's not so easy to see the convergence rate by looking at these numbers. But we can check the ratios of the log of successive errors. 

In [ ]:
logerr = @. log(abs(err))

In [ ]:
ratios = @. logerr[2:end] / logerr[1:end-1]

It seems to be heading toward a constant ratio of about 1.6 before it bumps up against machine precision.

# Example 4.5.2

Let us use Newton's method on the system defined by the function

In [ ]:
function nlvalue(x)
    return [ 
        exp(x[2]-x[1]) - 2
        x[1]*x[2] + x[3]
        x[2]*x[3] + x[1]^2 - x[2]
    ]
end

Here is a function that computes the Jacobian matrix.

In [ ]:
function nljac(x)
    J = zeros(3,3)
    J[1,:] = [-exp(x[2]-x[1]), exp(x[2]-x[1]), 0]
    J[2,:] = [x[2], x[1], 1]
    J[3,:] = [2*x[1], x[3]-1, x[2]]
    return J
end

(These functions could be written as separate files, or embedded within another function as we have done here.) Our initial guess at a root is the origin.

In [ ]:
x = [0,0,0]

We need the value (residual) of the nonlinear function, and its Jacobian, at this value for $\mathbf{x}$. 

In [ ]:
@show f = nlvalue(x)
J = nljac(x)

We solve for the Newton step and compute the new estimate. 

In [ ]:
s = -(J\f)
x = [x x[:,1]+s]

Here is the new residual.

In [ ]:
nlvalue(x[:,2])

We don't seem to be especially close to a root. Let's iterate a few more times. 

In [ ]:
for n = 2:7
    f = [f nlvalue(x[:,n])]
    s = -( nljac(x[:,n]) \ f[:,n] )
    x = [x x[:,n]+s]
end

In [ ]:
nlvalue(x[:,end])

In [ ]:
x[:,end]

We find the infinity norm of the residuals. 

In [ ]:
f

In [ ]:
residual_norm = maximum(abs.(f),dims=1)   # max in dimension 1

We don't know an exact answer, so we will take the last computed value as its surrogate. 

In [ ]:
r = x[:,end]
x = x[:,1:end-1]

The following will subtract r from every column of x.

In [ ]:
e = x .- r
e[:,2:end]

Now we take infinity norms and check for quadratic convergence. 

In [ ]:
errs = maximum(abs.(e),dims=1)
errs[2:end]'

In [ ]:
ratios = @. log(errs[2:end]) / log(errs[1:end-1])

For a brief time, we see ratios around 2, but then the limitation of double precision makes it impossible for the doubling to continue. 

# Example 4.5.3

As before, the system is defined by its residual and Jacobian, but this time we implement them as a single function.

In [ ]:
function nlsystem(x)
    f = [
        exp(x[2]-x[1]) - 2,
        x[1]*x[2] + x[3],
        x[2]*x[3] + x[1]^2 - x[2]
        ]
    J = [
        -exp(x[2]-x[1]) exp(x[2]-x[1]) 0;
        x[2] x[1] 1;
        2*x[1] x[3]-1 x[2]
    ]
    return f,J
end

Our initial guess is the origin. The output has one column per iteration.

In [ ]:
x1 = [0,0,0]
x = newtonsys(nlsystem,x1)

The last column contains the final Newton estimate. We'll compute the residual there in order to check the quality of the result.

In [ ]:
r = x[:,end]
f,J = nlsystem(r)
f

Let's use the convergence to the first component of the root as a proxy for the convergence of the vectors.

In [ ]:
@. log( 10,abs(x[1,1:end-1]-r[1]) )

The exponents approximately double, as is expected of quadratic convergence. 

# Example 4.6.1

To solve a nonlinear system, we need to code only the function defining the system (residual vector), and not its Jacobian.

In [ ]:
function nlsystem(x)
    return [ exp(x[2]-x[1]) - 2,
          x[1]*x[2] + x[3],
          x[2]*x[3] + x[1]^2 - x[2]
        ]
end

In all other respects usage is the same as for the `newtonsys` function. 

In [ ]:
x1 = [0,0,0]   
x = levenberg(nlsystem,x1)

It's always a good idea to check the accuracy of the root, by measuring the residual (backward error). 

In [ ]:
r = x[:,end]
@show backward_err = norm(nlsystem(r));

Looking at the convergence of the first component, we find a subquadratic convergence rate, just as with the secant method.

In [ ]:
@. log( 10, abs(x[1,1:end-1]-r[1]) )

# Example 4.7.1

Inhibited enzyme reactions often follow what are known as _Michaelis–Menten_ kinetics, in which a reaction rate $v$ follows a law of the form

$$v(x) = \frac{V x}{K_m + x},$$ 

where $x$ is the concentration of a substrate. The real values $V$ and $K_m$ are parameters that are free to fit to data. For this example we cook up some artificial data with $V=2$ and $K_m=1/2$.

In [ ]:
m = 25;
x = range(0.05,stop=6,length=m)
y = @. 2*x/(0.5+x)                   # exactly on the curve
@. y += 0.15*cos(2*exp(x/16)*x);     # noise added

In [ ]:
scatter(x,y,label="data",
     xlabel="x",ylabel="v",leg=:bottomright)

The idea is to pretend that we know nothing of the origins of this data and use nonlinear least squares on the misfit to find the parameters in the theoretical model function $v(x)$. Note in the Jacobian that the derivatives are _not_ with respect to $x$, but with respect to the two parameters, which are contained in the vector `c`.

In [ ]:
function misfit(c)
    V,Km = c   # rename components for clarity
    f = @. V*x/(Km+x) - y
    J = zeros(m,2)
    J[:,1] = @. x/(Km+x)              # d/d(V)
    J[:,2] = @. -V*x/(Km+x)^2         # d/d(Km)
    return f,J
end

In [ ]:
c1 = [1, 0.75]
c = newtonsys(misfit,c1)
@show V,Km = c[:,end]  # final values

The final values are close to the noise-free values of $V=2$, $K_m=0.5$ that we used to generate the data. We can calculate the amount of misfit at the end, although it's not completely clear what a "good" value would be. Graphically, the model looks reasonable.

In [ ]:
model = x -> V*x/(Km+x)
final_misfit_norm = norm(@.model(x)-y) 

In [ ]:
plot!(model,0,6,label="MM fit" )

For this model, we also have the option of linearizing the fit process. Rewrite the model as $1/v= (a/x)+b$ for the new parameters $\alpha=K_m/V$ and $\beta=1/V$. This corresponds to the misfit function whose entries are 

$$f_i(\alpha,\beta) = \alpha \cdot \frac{1}{x_i} + \beta - \frac{1}{y_i},$$ 

for $i=1,\ldots,m$. Although the misfit is nonlinear in $x$ and $y$, it's linear in the unknown parameters $\alpha$ and $\beta$, and so can be posed and solved as a linear least-squares problem.

In [ ]:
A = [ x.^(-1) x.^0 ];  u = @. 1/y;
z =  A\u;
alpha,beta = z

The two fits are different, because they do not optimize the same quantities. 

In [ ]:
linmodel = x -> 1 / (beta + alpha/x);
final_misfit_linearized = norm(@. linmodel(x)-y)

In [ ]:
plot!(linmodel,0,6,label="linearized fit")